In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
# dyf = glueContext.create_dynamic_frame.from_options(
#     connection_type="s3",
#     connection_options={
#         "paths": ["s3://ti-unique-bucket-12345/customers/"],
#         "recurse": True
#     },
#     format="csv",
#     format_options={
#         "withHeader": True,
#         "separator": ","
#     }
# )

# df = dyf.toDF()

In [17]:
df.show()

+---+----------+-----------+--------------------+
| id| firstname|   lastname|            fullname|
+---+----------+-----------+--------------------+
|293| Catherine|       Abel|      Catherine Abel|
|295|       Kim|Abercrombie|     Kim Abercrombie|
|297|  Humberto|    Acevedo|    Humberto Acevedo|
|291|   Gustavo|     Achong|      Gustavo Achong|
|299|     Pilar|   Ackerman|      Pilar Ackerman|
|305|     Carla|      Adams|         Carla Adams|
|301|   Frances|      Adams|       Frances Adams|
|307|       Jay|      Adams|           Jay Adams|
|309|    Ronald|      Adina|        Ronald Adina|
|311|    Samuel|   Agcaoili|     Samuel Agcaoili|
|313|     James|    Aguilar|       James Aguilar|
|315|    Robert|   Ahlering|     Robert Ahlering|
|319|       Kim|      Akers|           Kim Akers|
|441|   Stanley|       Alan|        Stanley Alan|
|323|       Amy|    Alberts|         Amy Alberts|
|325|      Anna|   Albright|       Anna Albright|
|327|    Milton|     Albury|       Milton Albury|


In [2]:
from pyspark.sql import SparkSession

# Create Spark session (Glue image already configures this)
spark = (
    SparkSession.builder
    .appName("ReadCustomersFromS3")
    .getOrCreate()
)

# S3 path
s3_path = "s3://ti-unique-bucket-12345/customers/"

# Read CSV from S3
df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(s3_path)
)

# Show data
df.show(truncate=False)
df.printSchema()




26/02/12 07:57:44 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


+---+----------+-----------+--------------------+
|id |firstname |lastname   |fullname            |
+---+----------+-----------+--------------------+
|293|Catherine |Abel       |Catherine Abel      |
|295|Kim       |Abercrombie|Kim Abercrombie     |
|297|Humberto  |Acevedo    |Humberto Acevedo    |
|291|Gustavo   |Achong     |Gustavo Achong      |
|299|Pilar     |Ackerman   |Pilar Ackerman      |
|305|Carla     |Adams      |Carla Adams         |
|301|Frances   |Adams      |Frances Adams       |
|307|Jay       |Adams      |Jay Adams           |
|309|Ronald    |Adina      |Ronald Adina        |
|311|Samuel    |Agcaoili   |Samuel Agcaoili     |
|313|James     |Aguilar    |James Aguilar       |
|315|Robert    |Ahlering   |Robert Ahlering     |
|319|Kim       |Akers      |Kim Akers           |
|441|Stanley   |Alan       |Stanley Alan        |
|323|Amy       |Alberts    |Amy Alberts         |
|325|Anna      |Albright   |Anna Albright       |
|327|Milton    |Albury     |Milton Albury       |


In [4]:
df.show()

+---+----------+-----------+--------------------+
| id| firstname|   lastname|            fullname|
+---+----------+-----------+--------------------+
|293| Catherine|       Abel|      Catherine Abel|
|295|       Kim|Abercrombie|     Kim Abercrombie|
|297|  Humberto|    Acevedo|    Humberto Acevedo|
|291|   Gustavo|     Achong|      Gustavo Achong|
|299|     Pilar|   Ackerman|      Pilar Ackerman|
|305|     Carla|      Adams|         Carla Adams|
|301|   Frances|      Adams|       Frances Adams|
|307|       Jay|      Adams|           Jay Adams|
|309|    Ronald|      Adina|        Ronald Adina|
|311|    Samuel|   Agcaoili|     Samuel Agcaoili|
|313|     James|    Aguilar|       James Aguilar|
|315|    Robert|   Ahlering|     Robert Ahlering|
|319|       Kim|      Akers|           Kim Akers|
|441|   Stanley|       Alan|        Stanley Alan|
|323|       Amy|    Alberts|         Amy Alberts|
|325|      Anna|   Albright|       Anna Albright|
|327|    Milton|     Albury|       Milton Albury|


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("iceberg") \
    .config("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.glue_catalog","org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.glue_catalog.catalog-impl","org.apache.iceberg.aws.glue.GlueCatalog") \
    .config("spark.sql.catalog.glue_catalog.warehouse","s3://ti-unique-bucket-12345/glue-data/") \
    .config("spark.sql.catalog.glue_catalog.io-impl","org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.defaultCatalog","glue_catalog") \
    .getOrCreate()


26/02/12 07:58:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [19]:
spark.sql("CREATE DATABASE bronze")


DataFrame[]

In [20]:
spark.sql("""
CREATE OR REPLACE TABLE glue_catalog.bronze.customers (
  ID INT,
  firstname STRING,
  lastname STRING,
  fullname STRING
)
USING ICEBERG
LOCATION 's3://ti-unique-bucket-12345/iceberg/customers'
""")


DataFrame[]

In [10]:
df.show()

+---+----------+-----------+--------------------+
| id| firstname|   lastname|            fullname|
+---+----------+-----------+--------------------+
|293| Catherine|       Abel|      Catherine Abel|
|295|       Kim|Abercrombie|     Kim Abercrombie|
|297|  Humberto|    Acevedo|    Humberto Acevedo|
|291|   Gustavo|     Achong|      Gustavo Achong|
|299|     Pilar|   Ackerman|      Pilar Ackerman|
|305|     Carla|      Adams|         Carla Adams|
|301|   Frances|      Adams|       Frances Adams|
|307|       Jay|      Adams|           Jay Adams|
|309|    Ronald|      Adina|        Ronald Adina|
|311|    Samuel|   Agcaoili|     Samuel Agcaoili|
|313|     James|    Aguilar|       James Aguilar|
|315|    Robert|   Ahlering|     Robert Ahlering|
|319|       Kim|      Akers|           Kim Akers|
|441|   Stanley|       Alan|        Stanley Alan|
|323|       Amy|    Alberts|         Amy Alberts|
|325|      Anna|   Albright|       Anna Albright|
|327|    Milton|     Albury|       Milton Albury|


In [21]:
df.writeTo("glue_catalog.bronze.customers").append()
